In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import platform
from sklearn.preprocessing import StandardScaler

from mabwiser.mab import MAB, LearningPolicy
from mabwiser.linear import _RidgeRegression, _Linear

class LinTSExample(_RidgeRegression):
    def predict(self, x):
        if self.scaler is not None:
            x = self._scale_predict_context(x) 
        covar = np.dot(self.alpha**2, self.A_inv)
        beta_sampled = rng.multivariate_normal(self.beta, covar)        
        return np.dot(x, beta_sampled)

    def init(self, num_features):
        # By default, assume that
        # A is the identity matrix and Xty is set to 0
        self.Xty = np.zeros(num_features)
#         rng2 = np.random.RandomState(seed=1)
#         perturbation = np.diag(rng2.standard_normal(num_features)) * 1e-10
#         self.A = self.l2_lambda * np.identity(num_features) + perturbation
        self.A = self.l2_lambda * np.identity(num_features)
        self.A_inv = np.linalg.inv(self.A)
        self.beta = np.dot(self.A_inv, self.Xty)
        
class LinearExample(_Linear):
    factory = {"ts": LinTSExample}

    def __init__(self, rng, arms, n_jobs=1, backend=None, l2_lambda=1, alpha=1, regression='ts', arm_to_scaler = None):
        super().__init__(rng, arms, n_jobs, backend, l2_lambda, alpha, regression)
       
        self.l2_lambda = l2_lambda
        self.alpha = alpha
        self.regression = regression

        # Create ridge regression model for each arm
        self.num_features = None

        if arm_to_scaler is None:
            arm_to_scaler = dict((arm, None) for arm in arms)

        self.arm_to_model = dict((arm, LinearExample.factory.get(regression)(rng, l2_lambda,
                                                                       alpha, arm_to_scaler[arm])) for arm in arms)


# Mac OS Big Sur OpenBLAS - Default

In [2]:
platform.platform()

'Darwin-21.5.0-x86_64-i386-64bit'

In [3]:
print(np.__version__)

1.18.5


In [4]:
users = pd.read_csv('movielens_users.csv')
responses = pd.read_csv('movielens_responses.csv')

In [5]:
train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user id')
context_features = [c for c in users.columns if c not in ['user id', 'set']]

decisions = MAB._convert_array(train['item id'])
rewards = MAB._convert_array(train['rated'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

scaler = pickle.load(open('movielens_scaler.pkl', 'rb'))

contexts = scaler.transform(contexts)
test_contexts = scaler.transform(test_contexts)

/Users/a649166/Tools/miniconda3/envs/dcb/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
rng = np.random.RandomState(seed=11)
arms = list(responses['item id'].unique())

mab = LinearExample(rng=rng, arms=arms, l2_lambda=10, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [7]:
mab.fit(decisions, rewards, contexts)
expectations = mab.predict_expectations(test_contexts)

expectations[0][1]

-0.03593678774678655

In [8]:
pickle.dump(mab, open(os.path.join('output', 'dar_ml_openblas_mab_default.pkl'), 'wb'))
pickle.dump(expectations, open(os.path.join('output', 'dar_ml_openblas_expectations_default.pkl'), 'wb'))

# Mac OS Big Sur OpenBLAS - Perturbed

In [9]:
class LinTSExample(_RidgeRegression):
    def predict(self, x):
        if self.scaler is not None:
            x = self._scale_predict_context(x) 
        covar = np.dot(self.alpha**2, self.A_inv)
        beta_sampled = rng.multivariate_normal(self.beta, covar)        
        return np.dot(x, beta_sampled)

    def init(self, num_features):
        # By default, assume that
        # A is the identity matrix and Xty is set to 0
        self.Xty = np.zeros(num_features)
        rng2 = np.random.RandomState(seed=1)
        perturbation = np.diag(rng2.standard_normal(num_features)) * 1e-10
        self.A = self.l2_lambda * np.identity(num_features) + perturbation
#         self.A = self.l2_lambda * np.identity(num_features)
        self.A_inv = np.linalg.inv(self.A)
        self.beta = np.dot(self.A_inv, self.Xty)
        
class LinearExample(_Linear):
    factory = {"ts": LinTSExample}

    def __init__(self, rng, arms, n_jobs=1, backend=None, l2_lambda=1, alpha=1, regression='ts', arm_to_scaler = None):
        super().__init__(rng, arms, n_jobs, backend, l2_lambda, alpha, regression)
       
        self.l2_lambda = l2_lambda
        self.alpha = alpha
        self.regression = regression

        # Create ridge regression model for each arm
        self.num_features = None

        if arm_to_scaler is None:
            arm_to_scaler = dict((arm, None) for arm in arms)

        self.arm_to_model = dict((arm, LinearExample.factory.get(regression)(rng, l2_lambda,
                                                                       alpha, arm_to_scaler[arm])) for arm in arms)


In [10]:
rng = np.random.RandomState(seed=11)
arms = list(responses['item id'].unique())

mab = LinearExample(rng=rng, arms=arms, l2_lambda=10, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [11]:
mab.fit(decisions, rewards, contexts)
expectations = mab.predict_expectations(test_contexts)

expectations[0][1]

-0.13434664072283034

In [12]:
pickle.dump(mab, open(os.path.join('output', 'dar_ml_openblas_mab_perturbed.pkl'), 'wb'))
pickle.dump(expectations, open(os.path.join('output', 'dar_ml_openblas_expectations_perturbed.pkl'), 'wb'))